In [1]:
# Copyright (C) 2024 co-pace GmbH (a subsidiary of Continental AG).
# Licensed under the BSD-3-Clause License.
# @author: Jonas Noah Michael Neuhöfer

## Animations

Here we generate animations to visualise the problem.

The rendered animations can be found in ``data/figures/``.

In [2]:
%load_ext autoreload
%autoreload 3
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.lines import Line2D
import matplotlib.patches as patches
from matplotlib.patches import PathPatch
from matplotlib.colors import LinearSegmentedColormap

import scipy
import scipy.stats as stats
import sys
    
from context import filters, visual, eval, utils
try:
    from tqdm import tqdm
    tqdm_pres = True
except:
    print("couldn't load tqdm")
    tqdm_pres = False
#plt.rcParams['animation.ffmpeg_path'] = os.path.abspath(os.path.join(os.path.abspath(''), '../STFenv/lib/python3.8/site-packages/ffmpeg'))
plt.rcParams.update(
    {
        "text.usetex": True,
        "text.latex.preamble": r"\usepackage{amsmath}\usepackage{amsfonts}\usepackage{bm}",
        "pdf.use14corefonts": True,
        
        'font.family': 'serif',
        'font.serif': ['Times'], #'Computer Modern Roman'
        #'mathtext.fontset': 'custom',
        #"mathtext.rm"  : "Computer Modern Roman",
        #"mathtext.it"  : "Computer Modern Roman:italic",
        #"mathtext.bf"  : "Computer Modern Roman:bold",
    }
)

seed = eval.simulate.make_seed(0)
skip = 4

- Loading package src
-  Loading package src/filters
-  Loading File 'src/utils.py'
-   Loading File 'abstract.py'
-   Loading File 'distributions.py'
-   Loading File 'models.py'
-   Loading File 'proposed.py'
-   Loading File 'robust.py'
-  Loading package src/visual
-  Loading package src/eval
-   Loading File 'simulate.py'
-   Loading File 'Distr_test.py'
-   Loading File 'Singer_test.py'
-   Loading File 'showcase.py'
-   Loading File 'animations.py'


In [3]:
%load_ext autoreload
%autoreload 3


model, (x0, P0), (x,y,v,e,t) = eval.simulate.simulate_Singer(seed=seed, T=200, beta=0.25)
x=x*np.array([1,0.75,1,1,1,1])[None,:,None]; y = y*np.array([1,0.75])[None,:,None]
dt = t[1]-t[0]
minx, miny, maxx, maxy = (x[:,0,0].min(), x[:,1,0].min(), x[:,0,0].max(), x[:,1,0].max())
d = max(maxx-minx, maxy-miny)
xlimmin, xlimmax, ylimmin, ylimmax = minx-0.05*d, maxx+0.05*d, miny-0.05*d, maxy+0.05*d
midx, midy = (minx+maxx)/2, (miny+maxy)/2
carlength = 0.05*d
#plt.rcParams.update( {'font.serif': ['Computer Modern Roman']} )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
if False:
    # This just tests that outliers out of frames are indicated correctly by arrows
    N = 20
    fig, ax = plt.subplots(1,1)
    inner_rect = patches.Rectangle((minx, miny), maxx-minx, maxy-miny, edgecolor='r', facecolor='none')
    lim_rect = patches.Rectangle((xlimmin, ylimmin), xlimmax-xlimmin, ylimmax-ylimmin, edgecolor='b', facecolor='none')
    ax.add_patch(inner_rect)
    ax.add_patch(lim_rect)
    view = np.array([minx/4+3*maxx/4, 4*miny/5+maxy/5])

    points = np.array([xlimmin-d/2, ylimmin-d/2])[None,:] + np.array([xlimmax-xlimmin+d, ylimmax-ylimmin+d])[None,:] * np.random.random(size=(N,2))
    ax.scatter([view[0]],[view[1]], s=4, color="r", marker="x",linewidths=.5)

    for i in range(N):
        ax.scatter([points[i,0]],[points[i,1]], s=2, color=f"C{i}")
        arrx,arry,arrdx,arrdy,out = visual.animations.arrow_for_outlier(view=view, obs=points[i],l=carlength, minx=minx, miny=miny, maxx=maxx, maxy=maxy)
        if out:
            arrow = ax.arrow(arrx,arry,arrdx,arrdy, width=carlength/4, head_length=carlength/2, head_width=carlength/2, fc=f"C{i}", ec="none", alpha=0.4, zorder=4)

    ax.set_xlim(xlimmin-d, xlimmax+d)
    ax.set_ylim(ylimmin-d, ylimmax+d)

In [5]:
upsampling_factor = 1
skip_y = 4  # how many observations should be skipped. Leads to a less cluttered image. Or smoother state updates between observations
away_pos = np.array([100,100,1,1,0,0])

upsampledx, upsampledt = visual.animations.upsampling(x=x[1:], t=t, model=model, dt=(t[1]-t[0])/upsampling_factor)
upsampledy, upsampledt_y = y[skip_y::skip_y,:,0], t[skip_y::skip_y]

e_norms_F = np.sqrt(np.sum(e[skip_y::skip_y,:,0]**2, axis=1)) # these are F(2,model._R_distr().nu) distributed, here F(2,1)
# we aim to make them chi-square chi2(2) distributed for the case of Gaussian noise instead of Students' t-distributed noise.
e_norms_chi = stats.chi2.ppf(stats.f.cdf(e_norms_F/2, 2, 0.75), 2)
upsampledy_Gaussian = x[1+skip_y::skip_y,:2,0]+(e_norms_chi/e_norms_F)[:,None]*e[skip_y::skip_y,:,0]

plt.rcParams['savefig.bbox'] = 'tight'
fig, ax  = plt.subplots(1, 1)
ax.set_xlim(xlimmin, xlimmax)
ax.set_ylim(ylimmin, ylimmax)
ax.set_aspect("equal")
ax.set_xticks([],[])
ax.set_yticks([],[])
fig.set_tight_layout(True)

#ax.plot(upsampledy[:,0], upsampledy[:,1], "bo", markersize=4)
car = visual.animations.Car(state=upsampledx[0,:,0], ax=ax, model=model, length=carlength, past=0, predict=0, fc="#333333", ec="none")

pbar = tqdm(desc="Animating only the agent", total=len(upsampledt), leave=None, position=0, file=sys.stdout)
def init1():
    car.keep_past(0)
    car.update(upsampledx[0,:,0])
    car.keep_past(200*upsampling_factor)
def animate_update1(frame):
    pbar.update()
    car.update(upsampledx[frame,:,0])

if True:
    ani = animation.FuncAnimation(fig, func=animate_update1, init_func=init1, frames=range(len(upsampledt)), repeat=True)
    Writer = animation.writers['ffmpeg']
    Writer = Writer(fps=30*upsampling_factor)
    ani.save(f"../data/figures/AnimationSimpleCar.mp4", writer=Writer, dpi=200)
plt.close(fig)

Animating only the agent:  99%|█████████▉| 199/201 [00:07<00:00, 26.07it/s]

In [6]:
fig, ax  = plt.subplots(1, 1)
ax.set_xlim(xlimmin, xlimmax)
ax.set_ylim(ylimmin, ylimmax)
ax.set_aspect("equal")
ax.set_xticks([],[])
ax.set_yticks([],[])
fig.set_tight_layout(True)


car = visual.animations.Car(state=upsampledx[1,:,0], ax=ax, model=model, length=carlength, past=0, predict=0, fc="#333333", ec="none", zorder=1)
obserCar = visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc="blue", ec="none", alpha=0.4, zorder=2)
obserCar2= visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc=[0,0,0.3], ec="none", alpha=0.4, zorder=3)
obserArrow = ax.arrow(*away_pos[:4], width=car.length/4, head_length=car.length/2, head_width=car.length/2, fc="blue", ec="none", alpha=0.4, zorder=4)

pbar = tqdm(desc="Animating the agent and observations", total=len(upsampledt), leave=None, position=0, file=sys.stdout)

def init2():
    init1()
    global last_ty
    last_ty = -1
    obserCar.update(away_pos)
    obserArrow.set_data(x=away_pos[0], y=away_pos[1])
    obserCar.keep_past(0)
    ax.legend([car.patch,obserCar.patch], ['ground truth position','noisy observations'], 
              handler_map={PathPatch: visual.animations.HandlerPathPatch()}, loc="upper right").set_visible(True)

def make_animate_update2(y_obs, times):
    def animate_update2(frame):
        car.update(upsampledx[frame,:,0])
        global last_ty
        if last_ty+1 < len(times) and upsampledt[frame]+1e-8 > times[last_ty+1]:
            last_ty += 1
            yx, yy = (y_obs[last_ty,0], y_obs[last_ty,1,])
            if yx < xlimmin or yx > xlimmax or yy < ylimmin or yy > ylimmax:
                x,y,dx,dy,out = visual.animations.arrow_for_outlier(view=upsampledx[frame,:2,0], obs=y_obs[last_ty],
                                                                    l=car.length, minx=minx, miny=miny, maxx=maxx, maxy=maxy)
                obserArrow.set_data(x=x,y=y,dx=dx,dy=dy)
                #ax.plot([upsampledx[frame,0,0], x, yx,], [upsampledx[frame,1,0], y, yy])
            else:
                obserArrow.set_data(x=away_pos[0], y=away_pos[1])
            obserCar.update([yx, yy, upsampledx[frame,2,0], upsampledx[frame,3,0], 0, 0])
        pbar.update()
    return animate_update2
if True:
    ani = animation.FuncAnimation(fig, func=make_animate_update2(upsampledy,upsampledt_y), init_func=init2, frames=range(len(upsampledt)), repeat=True)
    Writer = animation.writers['ffmpeg']
    Writer = Writer(fps=15*upsampling_factor)
    ani.save(f"../data/figures/AnimationStudentTNoise.mp4", writer=Writer, dpi=200)
plt.close(fig)

Animating the agent and observations: 100%|█████████▉| 200/201 [00:14<00:00, 15.38it/s]

In [7]:
fig, ax  = plt.subplots(1, 1)
ax.set_xlim(xlimmin, xlimmax)
ax.set_ylim(ylimmin, ylimmax)
ax.set_aspect("equal")
ax.set_xticks([],[])
ax.set_yticks([],[])
fig.set_tight_layout(True)

mode = 2 # can be 0 (Kalman Filtering), 1 (Kalman Gating) 2 (Our Proposed)

car = visual.animations.Car(state=upsampledx[0,:,0], ax=ax, model=model, length=carlength, past=0, predict=0, fc="#333333", ec="none", zorder=2)
obserCar = visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc="blue", ec="none", alpha=0.4, zorder=3)
filterCar= visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc="#C99F00", ec="none", zorder=1, alpha=0.5)
obserArrow = ax.arrow(*away_pos[:4], width=car.length/4, head_length=car.length/2, head_width=car.length/2, fc="blue", ec="none", alpha=0.4, zorder=4)
ellipse  = ax.plot(away_pos[:2], color="red", alpha=0.2, zorder=1)[0]
cancelX = ax.plot([],[], "x", mfc=[0.66,0,0], mec=[0.66,0,0], markersize=carlength*7.5, alpha=0.5, zorder=1.5)[0]

pbar = tqdm(desc=("Animating the Kalman Filter" if mode == 0 else ("Animating the gated Kalman Filter" if mode == 1 else "Animating our Method")), 
            total=len(upsampledt)*(2 if mode == 1 else 1), leave=None, position=0, file=sys.stdout)

def init3():
    init2()
    obserCar.update(away_pos)
    #car.keep_past(200*upsampling_factor)
    car.patch.set(visible = False)
    filterCar.update(x0)
    filterCar.keep_past(0)
    filterCar.keep_past(200*upsampling_factor)
    filterCar.patch.set(fc="#C99F00")
    filterCar.cmap = LinearSegmentedColormap.from_list('custom_cmap', ["#C99F00", "#C99F00",[1,1,1,0],])
    global filter_est
    filter_est = None
    #note.set_color([0,0,0,0])
def init3_our():
    init3()
    filterCar.patch.set(fc="#ff4500")
    filterCar.cmap = LinearSegmentedColormap.from_list('custom_cmap', ["#ff4500", "#ff4500",[1,1,1,0],])
    #filterCar.keep_past(filterCar.past)
    #note.set_text("our filter")
    #note.set_color("black")

def make_init3(inmode):
    if inmode == -1:
        def init3_new():
            init3()
            ax.legend([Line2D([],[],color="black"), obserCar.patch,filterCar.patch], 
                    ['ground truth position','Gaussian observations','Kalman Filter'], 
                    handler_map={PathPatch: visual.animations.HandlerPathPatch()}, loc="upper right").set_visible(True)
    elif inmode == 0:
        def init3_new():
            init3()
            ax.legend([Line2D([],[],color="black"), obserCar.patch,filterCar.patch], 
                    ['ground truth position','noisy observations','Kalman Filter'], 
                    handler_map={PathPatch: visual.animations.HandlerPathPatch()}, loc="upper right").set_visible(True)
    elif inmode == 1:
        def init3_new():
            init3()
            ax.legend([Line2D([],[],color="black"), obserCar.patch, filterCar.patch, Line2D([],[],color="red",alpha=0.2)], 
                    ['ground truth position','noisy observations','gated Kalman Filter', 'discarding boundary'], 
                    handler_map={PathPatch: visual.animations.HandlerPathPatch()}, loc="upper right").set_visible(True)
    else:
        def init3_new():
            init3_our()
            filterCar.patch.set(fc="#ff4500")
            ax.legend([Line2D([],[],color="black"), obserCar.patch, filterCar.patch,], 
                    ['ground truth position','noisy observations','proposed method'], 
                    handler_map={PathPatch: visual.animations.HandlerPathPatch()}, loc="upper right").set_visible(True)
    return init3_new

def make_animate_update3(y_obs, times, filter, draw_ellipse_p=None, elongate=1, **filterkwargs): # upsampledy, upsampledt_y
    prev_update2 = make_animate_update2(y_obs, times)
    F_matrices = {}
    def F(dt):
        if dt not in F_matrices:
            F_matrices[dt] = model._F(dt)
        return F_matrices[dt]
    Q_matrices = {}
    def Q(dt):
        if dt not in Q_matrices:
            Q_matrices[dt] = model._Q(dt)
        return Q_matrices[dt]
    def animate_update3(frame):
        frame = int(frame/elongate)
        prev_update2(frame)
        global filter_est
        global filtered_t
        if filter_est == None:
            filter_est, filtered_t = filter(model=model, mean=x0, covar=P0, **filterkwargs).filter(y_obs,times).get_all_state_distr()
            filterCar.update(filter_est.mean()[0,:,0])
            filterCar.keep_past(int(200*upsampling_factor*elongate))
            car.keep_past(int(200*upsampling_factor*elongate))
            #if elongate-1 > 1e-8:
            #   note.set_text(f"{elongate:.2f}x slower")
            #   note.set_color("black")
            #elif elongate-1 < -1e-8:
            #   note.set_text(f"{1/elongate:.2f}x faster")
            #   note.set_color("black")
        mean_last, covar_last = filter_est.mean()[last_ty,:,0], filter_est.cov()[last_ty,:,:]
        filter_dt = upsampledt[frame] - times[last_ty] if last_ty > 0 and upsampledt[frame]+1e-8 > times[last_ty] else 0
        mean_pred = F(filter_dt) @ filter_est.mean()[last_ty+1,:,0]
        if draw_ellipse_p is not None and (frame > 0 and last_ty >= 0 and upsampledt[frame-1]+1e-8 < times[last_ty]):
            pred_dt = filtered_t[last_ty+1]-filtered_t[last_ty]
            covar_pred = (F(pred_dt)@covar_last@F(pred_dt).T +Q(pred_dt))[:2,:2] + model._R #causes epilepsy
            ellipse_data = utils.confidence_ellipse(covar_pred,p=draw_ellipse_p)(np.linspace(0,1,1001))
            # need to do some weird shit because the sensor points are at the front-wheels (which might be outside the ellipse when the midpoint is not)
            car_ori = np.abs(obserCar.rear_pos[:2]-obserCar.front_pos[:2])/obserCar.length
            ellipse_scaling = (np.max(ellipse_data,axis=1)-car_ori)/np.max(ellipse_data,axis=1)
            ellipse_scaling = np.array([max(ellipse_scaling[0],0.5),max(ellipse_scaling[1],0.5)])[:,None]
            ellipse_data *= ellipse_scaling
            ellipse.set_data(ellipse_data+mean_pred[:2,None] )
            if 'gating' in filterkwargs:
                res = y_obs[last_ty] - (F(pred_dt)@mean_last)[:2]
                gamma = res.T @ scipy.linalg.solve(covar_pred, res, check_finite=False, assume_a='pos')
                if gamma > scipy.stats.chi2(2).ppf(filterkwargs['gating']):
                    obserposx, obserposy = (obserCar.rear_pos[:2]+obserCar.front_pos[:2])/2
                    if obserposx < xlimmin or obserposx > xlimmax or obserposy < ylimmin or obserposy > ylimmax:
                        obserposx, obserposy, dx, dy, out = visual.animations.arrow_for_outlier(
                                obs=obserCar.front_pos[:2], view=upsampledx[frame,:2,0],
                                l=car.length, minx=minx, miny=miny, maxx=maxx, maxy=maxy  )
                        obserposx += dx; obserposy += dy
                    #ax.plot([upsampledx[frame,0,0],obserposx],[upsampledx[frame,1,0],obserposy], "--")
                    cancelX.set_data([obserposx], [obserposy])
                else:
                    cancelX.set_data([],[])
        filterCar.update(mean_pred)

    return animate_update3
if True:
    if mode == 0:
        ani = animation.FuncAnimation(fig, func=make_animate_update3(upsampledy,upsampledt_y,filters.basic.KalmanFilter), init_func=make_init3(mode), frames=range(len(upsampledt)), repeat=True)
    elif mode == 1:
        ani = animation.FuncAnimation(fig, func=make_animate_update3(upsampledy,upsampledt_y,filters.basic.KalmanFilter,draw_ellipse_p=0.99,elongate=2,gating=0.99), init_func=make_init3(mode), frames=range(len(upsampledt)*2), repeat=True)
    elif mode == 2:
        ani = animation.FuncAnimation(fig, func=make_animate_update3(upsampledy,upsampledt_y,filters.proposed.StudentTFilter, nu=3), init_func=make_init3(mode), frames=range(len(upsampledt)), repeat=True)
    Writer = animation.writers['ffmpeg']
    Writer = Writer(fps=20*upsampling_factor)
    ani.save(f"../data/figures/AnimationMethod{mode}.mp4", writer=Writer, dpi=200)
plt.close(fig)

Animating our Method: 100%|█████████▉| 200/201 [00:16<00:00, 12.50it/s]

In [8]:
filter_class_kwargs_GT=[
    (filters.proposed.StudentTFilter_GT, {"nu":3}, lambda x,y,v,e,t: {"GTx":x, "GTe":e}),
    (filters.proposed.StudentTFilter, {"nu":3}, lambda x,y,v,e,t: {}),
    (filters.robust.Huang_SSM, {"separate":True, "SSM":"log", "nu_SSM":3, "process_non_gaussian": True, "sigma_SSM": 1, "gating":1}, lambda x,y,v,e,t: {}),
    (filters.robust.Agamennoni_VBF, {"nu":8, "gating":1}, lambda x,y,v,e,t: {}),
    (filters.robust.chang_RKF, {"alpha":0.01}, lambda x,y,v,e,t: {}),
    (filters.robust.chang_ARKF, {"alpha":0.01}, lambda x,y,v,e,t: {}),
    (filters.robust.Saerkkae_VBF, {"rho":0, "gating":1}, lambda x,y,v,e,t: {}),
    (filters.robust.roth_STF, {"state_nu":5, "process_gamma":11, "obs_delta":2.5}, lambda x,y,v,e,t: {}),
    (filters.basic.KalmanFilter, {}, lambda x,y,v,e,t: {"gating":0.99}),
]
colors = ['#800000', '#ff4500', '#000075', '#006400', '#4363d8', '#42d4f4', '#00ff00', '#f58231', '#9a9a9a',]

In [9]:
def init_multiple_filters():
    init3()
    ax.legend([Line2D([],[],color="black")],["nothing"]).set_visible(False)
    filterCar.update(away_pos)
    filterCar.keep_past(0)
    car.patch.set_visible(True)
    car.patch.set(alpha=0.75,fc="black")
    global filter_cars, filter_ests
    try:
        for i in range(len(filter_cars)):
            filter_cars[i].patch.set(visible = False)
            filter_cars[i]._past.set(visible = False)
    except Exception as e:
        print("Failed to remove car collection")
        print(e)
        pass

    filter_cars = None; filter_ests = None

multiple_filters_seed = seed
def make_animate_multiple_filters(elongate=1):

    F_matrices = {}
    def F(dt):
        if dt not in F_matrices:
            F_matrices[dt] = model._F(dt)
        return F_matrices[dt]

    def animate_multiple_filters(frame):
        frame = int(frame/elongate)
        global filter_cars, filter_ests
        global _upsampledx, _upsampledt, _upsampledy, _upsampledt_y, _GTx, _GTy, _GTe, _GTv, _GTt
        global last_ty, multiple_filters_seed
        is_defined = False
        try:
            _upsampledt_y
            is_defined = True
        except Exception:
            pass
        if is_defined and last_ty+1 < len(_upsampledt_y) and _upsampledt[frame]+1e-8 > _upsampledt_y[last_ty+1]:
            last_ty += 1
            yx, yy = (_upsampledy[last_ty,0], _upsampledy[last_ty,1,])
            if yx < xlimmin or yx > xlimmax or yy < ylimmin or yy > ylimmax:
                x,y,dx,dy,out = visual.animations.arrow_for_outlier(view=_upsampledx[frame,:2,0], obs=_upsampledy[last_ty,:2],
                                            l=car.length, minx=minx, miny=miny, maxx=maxx, maxy=maxy)
                obserArrow.set_data(x=x,y=y,dx=dx,dy=dy)
                #ax.plot([upsampledx[frame,0,0], x, yx,], [upsampledx[frame,1,0], y, yy])
            else:
                obserArrow.set_data(x=away_pos[0], y=away_pos[1])
            obserCar.update([yx, yy, _upsampledx[frame,2,0], _upsampledx[frame,3,0], 0, 0])

        if filter_ests == None:
            # At the first frame
            _model, (_x0, _P0), (_x,_y,_v,_e,_t) = eval.simulate.simulate_Singer(seed=multiple_filters_seed, T=200, beta=0.25)
            multiple_filters_seed = eval.simulate.make_seed(multiple_filters_seed)
            multiple_filters_seed = eval.simulate.make_seed(multiple_filters_seed)
            _x=_x*np.array([1,0.75,1,1,1,1])[None,:,None]; _y = _y*np.array([1,0.75])[None,:,None]
            _upsampledx, _upsampledt = visual.animations.upsampling(x=_x[1:], t=_t, model=_model, dt=(_t[1]-_t[0])/upsampling_factor)
            _upsampledy, _upsampledt_y = _y[skip_y::skip_y,:,:], _t[skip_y::skip_y]
            _upsampledy = _upsampledy[:,:,0]

            _minx, _miny, _maxx, _maxy = (_x[:,0,0].min(), _x[:,1,0].min(), _x[:,0,0].max(), _x[:,1,0].max())
            x_trans = lambda xpos: (xpos-_minx)/(_maxx-_minx)*(maxx-minx)+minx
            y_trans = lambda ypos: (ypos-_miny)/(_maxy-_miny)*(maxy-miny)+miny
            def trans(pos):
                pos[:,0] = (pos[:,0]-_minx)/(_maxx-_minx)*(maxx-minx)+minx
                pos[:,1] = (pos[:,1]-_miny)/(_maxy-_miny)*(maxy-miny)+miny
            #_GTx, _GTy, _GTe, _GTv, _GTt = _x[skip_y::skip_y,:,:],_upsampledy,_v[skip_y::skip_y,:,:],_e[skip_y::skip_y,:,:],_upsampledt_y
            #_x0[0] = x_trans(_x0[0]); _x0[1] = x_trans(_x0[1]); 
            #for _GT in [_GTx, _GTy, _GTe, _GTv, _upsampledx]:
            #    _GT[:,0,:] = x_trans(_GT[:,0,:]); _GT[:,1,:] = y_trans(_GT[:,1,:])
            
            filter_ests = []; filter_cars = []
            for i in range(len(filter_class_kwargs_GT)):
                filter_est, filtered_t = filter_class_kwargs_GT[i][0](
                        model=_model, mean=_x0, covar=_P0, **filter_class_kwargs_GT[i][1], 
                        **(filter_class_kwargs_GT[i][2](_x[skip_y::skip_y,:,:],_y[skip_y::skip_y,:,:],_v[skip_y::skip_y,:,:],_e[skip_y::skip_y,:,:],_t[skip_y::skip_y]))
                    ).filter(_upsampledy,_upsampledt_y).get_all_state_distr()
                filter_ests.append(filter_est.mean())
                trans(filter_ests[i])
                filter_cars.append(visual.animations.Car(
                        state=_x0, ax=ax, model=model, length=carlength, past=0, 
                        predict=0, fc=colors[i], ec="none"
                    ))
            trans(_upsampledx)
            trans(_upsampledy)
            for i in range(len(filter_class_kwargs_GT)):
                filter_cars[i].update(filter_ests[i][0,:,0])
                filter_cars[i].keep_past(0)
                filter_cars[i].keep_past(int(40*upsampling_factor*elongate))
                filter_cars[i].patch.set(visible = False)
                car.update(_upsampledx[frame,:,0])
                car.keep_past(0)
                car.keep_past(int(40*upsampling_factor*elongate))
            if elongate-1 > 1e-8:
                note.set_text(f"{elongate:.2f}x slower")
                note.set_color("black")
            elif elongate-1 < -1e-8:
                note.set_text(f"{1/elongate:.2f}x faster")
                note.set_color("black")
            
        car.update(_upsampledx[frame,:,0])
        filter_dt = upsampledt[frame] - _upsampledt_y[last_ty] if last_ty > 0 and upsampledt[frame]+1e-8 > _upsampledt_y[last_ty] else 0
        for i in range(len(filter_class_kwargs_GT)):
            filter_cars[i].update(F(filter_dt) @ filter_ests[i][last_ty+1,:,0])
            
        pbar.update()

    return animate_multiple_filters


fig, ax  = plt.subplots(1, 1, figsize=(10,6))
ax.set_xlim(xlimmin, xlimmax)
ax.set_ylim(ylimmin, ylimmax)
ax.set_aspect("equal")
#ax.set_xticks([],[])
#ax.set_yticks([],[])

car = visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc="#333333", ec="none")
obserCar = visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc="blue", ec="none", alpha=0.4)
filterCar= visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc="#9a9a9a", ec="none")#, alpha=0.4)
obserArrow = ax.arrow(*away_pos[:4], width=car.length/4, head_length=car.length/2, head_width=car.length/2, fc="blue", ec="none", alpha=0.4)
ellipse = ax.plot(away_pos[:2], color="red", alpha=0.2)[0]
cancelX = ax.plot([],[], "x", mfc=[0.66,0,0], mec=[0.66,0,0], markersize=carlength*7.5, alpha=0.5)[0]

comment = ax.text(x=midx, y=midy, ha="center", va="center", fontsize=22, s="")
note = ax.text(x=maxx, y=maxy, ha="right", va="top", fontsize=14, s=f"")


if True:
    pbar = tqdm(desc="Animating Multiple robust filters", total=len(upsampledt), leave=None, position=0, file=sys.stdout)

    ani = animation.FuncAnimation(fig, func=make_animate_multiple_filters(), init_func=init_multiple_filters, frames=range(len(upsampledt)), repeat=True)
    Writer = animation.writers['ffmpeg']
    Writer = Writer(fps=30*upsampling_factor)
    ani.save(f"../data/figures/AnimationMultipleFilters.mp4", writer=Writer, dpi=200)
plt.close(fig)



Animating our Method: 100%|██████████| 201/201 [00:17<00:00, 11.62it/s]/s]
Failed to remove car collection
name 'filter_cars' is not defined
Animating Multiple robust filters: 100%|██████████| 201/201 [00:16<00:00, 13.21it/s]

In [10]:
blend_in__frames = 30*upsampling_factor
blend_out_frames = 30*upsampling_factor
stay_frames = 90*upsampling_factor
show_comment_frames = blend_in__frames + blend_out_frames + stay_frames + 2

def init_comment():
    ax.set_visible(True)
    car.update(away_pos)
    car.keep_past(0)
    obserCar.update(away_pos)
    obserCar.keep_past(0)
    filterCar.update(away_pos)
    filterCar.keep_past(0)
    obserArrow.set_data(x=away_pos[0], y=away_pos[1], dx=away_pos[2], dy=away_pos[3])
    ellipse.set_data([],[])
    comment.set_fontsize(22)
    comment.set_color([0,0,0,0])
    note.set_color([0,0,0,0])
    cancelX.set_data([], [])
    ax_time.set_visible(False); ax_error.set_visible(False)
    ax.legend([Line2D([],[],color="black")],["nothing"]).set_visible(False)

def init_qualitative():
    init_comment()
    ax.set_visible(False)
    ax_time.set_visible(True); ax_error.set_visible(True)

def empty(frame):
    pbar.update()

def init_Title():
    init_comment()
    comment.set_text("$\\textbf{An Outlier-Robust and Efficient}$\n$\\textbf{Bayesian Filter}$")
def init_FreeAgent():
    init_comment()
    comment.set_text("Imagine a highly maneuverable,\nfreely moving agent")
def init_Tracking_Gaus():
    init_comment()
    comment.set_text("When \\textbf{tracking} the agent from afar,\nonly \\textbf{noisy measurements} of it\nare available") # spacing?
def init_KF_intro05():
    init_comment()
    comment.set_text("We want to \\textbf{reconstruct} the trajectory\nonly from these noisy measurements")
def init_KF_intro():
    init_comment()
    comment.set_text("If this noise is \\textbf{Gaussian}, then the\n\\textbf{Kalman Filter} finds \\textbf{optimal}\nestimates \\textbf{efficiently} ")
def init_KF_outliers():
    init_comment()
    comment.set_text("However, \\textbf{real-world} measurements are\noften \\textbf{non-Gaussian} and contain,\nfor example, \\textbf{outliers}")
def init_Gating1():
    init_comment()
    comment.set_text("To \\textbf{counteract} outliers, a common\npractice is to try to \\textbf{classify} and\nsubsequently \\textbf{ignore} them")
def init_Gating2():
    init_comment()
    comment.set_text("This '\\textbf{Gating}' declares measurements\noutside a \\textbf{confidence ellipse} around\nthe \\textbf{hypothesis} as outliers")
def init_Gating3():
    init_comment()
    comment.set_text("However, this may \\textbf{discard}\n\\textbf{valid measurements}")
def init_Proposed1():
    init_comment()
    #comment.set_text("\\textbf{Prior work} has modelled such non-Gaussian\nmeasurements with \\textbf{more general} multivariate\n\\textbf{Student-t} distributions") #better spacing?
    comment.set_text("In \\textbf{prior work}, such non-Gaussian\nmeasurements are modeled with\nmore general multivariate\n\\textbf{Student-t} distributions") #three lines?
def init_Proposed2():
    init_comment()
    comment.set_text("Our proposed filter and smoother\nare leveraging the following equality\nof Student-t densities $t_{\\omega }(x|\\mu, \\Sigma)$")
    #comment.set_text("We leverage \\textbf{properties} of the Student-t\ndistribution directly to propose a \\textbf{robust} and\n\\textbf{efficient} \\textbf{filter} and \\textbf{smoother}")
def init_Proposed3():
    init_comment()
    comment.set_text("$t_{\\omega }(e|0, R) \\cdot t_{\\omega +m} \\left( x | \\mu, P\\right) \\cdot  t_{\\omega +m+n} \\left( v | 0, Q\\right)$"
                     +"\n"+"$= t_{\\omega }   \\left(   \\left[    \\begin{array}{c} x\\\\v\\\\e \\end{array}    \\right]\\right|  \\left[    \\begin{array}{c} \\mu\\\\0\\\\0 \\end{array}    \\right] ,\\left. \\left[    \\begin{array}{ccc} a(e) P&           0  &  0\\\\0&        a(e) b(x) Q &  0 \\\\ 0&           0  &  R \\end{array}    \\right]  \\right)$"
                     +"\n\nwith\n"+"$a(e) = \\tfrac{\\omega +m}{\\omega +e^\\top R^{-1}e}$ and $b(x) = \\tfrac{\\omega +m+n}{\\omega +m+ (x-\\mu)^{ \\top}  P^{-1}(x-\\mu)}$")
    comment.set_fontsize(18)
def init_Proposed4():
    init_comment()
    comment.set_text("This allows for \\textbf{localised} approximations\nof joint Student-t distributions")
def init_Proposed5():
    init_comment()
    comment.set_text("Our methods then arise from other\nbasic mathematical properties of\nStudent-t distributions")
def init_Multiple():
    init_comment()
    comment.set_text("We further compare our methods with\ncomparable \\textbf{state-of-the-art} methods")
def init_Quali():
    global filter_cars, filter_ests
    try:
        for i in range(len(filter_cars)):
            filter_cars[i].patch.set(visible = False)
            filter_cars[i]._past.set(visible = False)
    except Exception as e:
        print("Failed to remove car collection")
        print(e)
        pass
    init_comment()
    comment.set_text("We further summarise \\textbf{computation time}\nand \\textbf{performance} of the different methods\nover 5000 simulations")
def init_Quali2():
    init_comment()
    comment.set_text("The leftmost maroon method showcases\nthe performance gain better approximations\nof $a(e_k)$ and $b(x_k)$ could bring")

def make_show_comment(more_stay_frames=0):
    def show_comment(frame):
        if frame <= blend_in__frames:
            comment.set_color([0,0,0,frame/blend_in__frames])
        elif frame > blend_in__frames+stay_frames+more_stay_frames+1:
            comment.set_color([0,0,0,(blend_out_frames-frame+blend_in__frames+stay_frames+more_stay_frames+1)/blend_out_frames])
        pbar.update()
    return show_comment
def make_show_Title(more_stay_frames=0):
    def show_Title(frame):
        if frame <= blend_in__frames:
            comment.set_color([0,0,0,frame/blend_in__frames])
            authors.set_color([0,0,0,frame/blend_in__frames])
        elif frame > blend_in__frames+stay_frames+more_stay_frames+1:
            comment.set_color([0,0,0,(blend_out_frames-frame+blend_in__frames+stay_frames+more_stay_frames+1)/blend_out_frames])
            authors.set_color([0,0,0,(blend_out_frames-frame+blend_in__frames+stay_frames+more_stay_frames+1)/blend_out_frames])
        pbar.update()
    return show_Title

fig, ax  = plt.subplots(1, 1, figsize=(10,6))
ax.set_xlim(xlimmin, xlimmax)
ax.set_ylim(ylimmin, ylimmax)
ax.set_aspect("equal")
ax.set_xticks([],[])
ax.set_yticks([],[])
ax_time, ax_error, ax_inv1, ax_inv2 = fig.add_subplot(5,2,(3,7)), fig.add_subplot(5,2,(4,8)), fig.add_subplot(2,2,1), fig.add_subplot(2,2,2)
ax_inv1.set_visible(False); ax_inv2.set_visible(False)
ax_time.set_xticks(np.arange(9),['our, oracle\n$a(e)$, $b(x)$', 'our', 'Huang 2020', 'Agamennoni 2011', 'Chang 2014', 'Chang 2014', 'Särkkä 2012', 'Roth 2013', 'Kalman Filter'], rotation=-90)
ax_error.set_xticklabels(['our, oracle\n$a(e)$, $b(x)$', 'our', 'Huang 2020', 'Agamennoni 2011', 'Chang 2014', 'Chang 2014', 'Särkkä 2012', 'Roth 2013', 'Kalman Filter'], rotation=-90)

comment = ax.text(x=midx, y=midy, ha="center", va="center", fontsize=22, s="")
authors = ax.text(x=midx, y=(midy+miny)/2, ha="center", va="center", fontsize=14, s="Jonas Neuhöfer \\hspace{10pt} Jörg Reichardt")
authors.set_color([0,0,0,0])
init_Proposed3()
#ax_time.set_visible(True); ax_error.set_visible(True)
comment.set_color([0,0,0,1])
plt.close(fig)

/tmp/ipykernel_80853/2445542904.py:125: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax_error.set_xticklabels(['our, oracle\n$a(e)$, $b(x)$', 'our', 'Huang 2020', 'Agamennoni 2011', 'Chang 2014', 'Chang 2014', 'Särkkä 2012', 'Roth 2013', 'Kalman Filter'], rotation=-90)


In [11]:
# collect data of the quantitative analysis
model_kwargs  = {"alpha":0.5, "beta":0.25, "T":500, "R_nu":1} # TODO: needs to be rerendered with the proper values
best_results = eval.simulate.collect_best_results(model_kwargs, eval.simulate.Filters_To_Optimise)

for <class 'src.filters.proposed.StudentTFilter_GT'>: {}
for <class 'src.filters.proposed.StudentTFilter'>: {'nu': 3.2644}
for <class 'src.filters.robust.Huang_SSM'>: {'SSM': 'log', 'nu_SSM': 2.6022000000000003}
for <class 'src.filters.robust.chang_RKF'>: {'alpha': 0.0883}
for <class 'src.filters.robust.Agamennoni_VBF'>: {'nu': 12.3095}
for <class 'src.filters.basic.KalmanFilter'>: {'gating': 0.9999}
for <class 'src.filters.robust.chang_ARKF'>: {'alpha': 0.1764}
for <class 'src.filters.robust.Saerkkae_VBF'>: {'rho': 0.0001}
for <class 'src.filters.robust.roth_STF'>: {'obs_delta': 2.1737, 'process_gamma': 12.387500000000001, 'state_nu': 4.404100000000001}


In [12]:
concatenate = [
    (init_Title, make_show_Title(), (0, show_comment_frames), 
     "Display title                                 "),
    (init_FreeAgent, make_show_comment(-30*upsampling_factor), (0, show_comment_frames-30*upsampling_factor), 
     "Display setting                               "),
    (init1, animate_update1, (0,len(upsampledt)), 
     "Animating only the agent                      "),
    (init_Tracking_Gaus, make_show_comment(), (0, show_comment_frames), 
     "Display sensor problem                        "),
    (init2, make_animate_update2(upsampledy_Gaussian,upsampledt_y), (0,len(upsampledt)), 
     "Animating the agent and Gaussian observations "),
    (init_KF_intro05, make_show_comment(), (0, show_comment_frames), 
     "Display reconstruction task                   "),
    (init_KF_intro, make_show_comment(30*upsampling_factor), (0, show_comment_frames+30*upsampling_factor), 
     "Display Kalman Filter intro                   "),
    (make_init3(-1), make_animate_update3(upsampledy_Gaussian,upsampledt_y,filters.basic.KalmanFilter), (0,len(upsampledt)), 
     "Animating the Kalman Filter                   "),
    (init_KF_outliers, make_show_comment(30*upsampling_factor), (0, show_comment_frames+30*upsampling_factor), 
     "Display real-worl measurements non-Gaussian   "),
    (make_init3(0), make_animate_update3(upsampledy,upsampledt_y,filters.basic.KalmanFilter), (0,len(upsampledt)), 
     "Animating the agent and Student-t observations"),
    (init_Gating1, make_show_comment(30*upsampling_factor), (0, show_comment_frames+30*upsampling_factor), 
     "Display gating explanation 1                  "),
    (init_Gating2, make_show_comment(15*upsampling_factor), (0, show_comment_frames+15*upsampling_factor), 
     "Display gating explanation 2                  "),
    (init_Gating3, make_show_comment(), (0, show_comment_frames), 
     "Display gating problem                        "),
    (make_init3(1), make_animate_update3(upsampledy,upsampledt_y,filters.basic.KalmanFilter, draw_ellipse_p=0.99, elongate=2, gating=0.99), (0,int(len(upsampledt)*2)), 
     "Animating Gating                              "),
    (init_Proposed1, make_show_comment(30*upsampling_factor), (0, show_comment_frames+30*upsampling_factor), 
     "Display prior work                            "),
    (init_Proposed2, make_show_comment(30*upsampling_factor), (0, show_comment_frames+30*upsampling_factor), 
     "Display our approach                          "),
    (init_Proposed3, make_show_comment(30*upsampling_factor), (0, show_comment_frames+30*upsampling_factor), 
     "Display equalities                            "),
    (init_Proposed4, make_show_comment(), (0, show_comment_frames), 
     "Display local approximations                  "),
    (init_Proposed5, make_show_comment(), (0, show_comment_frames), 
     "Display mathematical properties               "),
    (make_init3(2), make_animate_update3(upsampledy,upsampledt_y,filters.proposed.StudentTFilter, nu=3), (0,len(upsampledt)), 
     "Animating our method                          "),
    (init_Multiple, make_show_comment(), (0, show_comment_frames), 
     "Display qualitative comparison                "),
    (init_multiple_filters, make_animate_multiple_filters(), (0,len(upsampledt)), 
     "Display comparison on familiar trajectory     "),
    (init_multiple_filters, make_animate_multiple_filters(), (0,int(len(upsampledt))), 
     "Display comparison on trajectory 2            "),
#    (init_multiple_filters, make_animate_multiple_filters(), (0,int(len(upsampledt))), 
#     "Display comparison on trajectory 3            "),
    (init_Quali, make_show_comment(30*upsampling_factor), (0, show_comment_frames+30*upsampling_factor), 
     "Display comparison with other methods         "),
    (init_qualitative, empty, (0,len(upsampledt)+60*upsampling_factor), 
     "Show qualitative results                      "),
    (init_Quali2, make_show_comment(), (0, show_comment_frames), 
     "Display qualitative comparison                "),
    (init_qualitative, empty, (0,len(upsampledt)+60*upsampling_factor), 
     "Show qualitative results                      "),
]
tot_len = sum([c[2][1]-c[2][0]  for c in concatenate])
pbar = tqdm(desc="", total=tot_len, leave=None, position=0, file=sys.stdout)
def animate_update_concatenate(frame):
    i = 0
    while frame >= concatenate[i][2][1]-concatenate[i][2][0]:
        frame -= concatenate[i][2][1]-concatenate[i][2][0]
        i += 1
    if frame == 0:
        concatenate[i][0]()
        pbar.set_description(concatenate[i][3])
    concatenate[i][1](frame+concatenate[i][2][0])

fig, ax  = plt.subplots(1, 1, figsize=(10,6))
ax.set_xlim(xlimmin, xlimmax)
ax.set_ylim(ylimmin, ylimmax)
ax.set_aspect("equal")
ax.set_xticks([],[])
ax.set_yticks([],[])
ax_time, ax_error, ax_inv1, ax_inv2 = fig.add_subplot(5,2,(3,7)), fig.add_subplot(5,2,(4,8)), fig.add_subplot(2,2,1), fig.add_subplot(2,2,2)
ax_inv1.set_visible(False); ax_inv2.set_visible(False)
ax_time.set_visible(False); ax_error.set_visible(False)
eval.simulate.make_boxplot(results=best_results, ax=ax_time, categories="comp_time_filter", maxboxoutlier=5)
eval.simulate.make_boxplot(results=best_results, ax=ax_error, categories="euc_error")
ax_time.set_ylim(0, 0.5*ax_time.get_ylim()[1]); ax_error.set_ylim(0, ax_error.get_ylim()[1])
ax_time.set_xticklabels(['our, oracle\n$a(e)$, $b(x)$', 'our', 'Huang 2020', 'Chang 2014', 'Agamennoni 2011', 'gated\nKalman Filter', 'Chang 2014', 'Särkkä 2012', 'Roth 2013'], rotation=-90)
ax_error.set_xticklabels(['our, oracle\n$a(e)$, $b(x)$', 'our', 'Huang 2020', 'Chang 2014', 'Agamennoni 2011', 'gated\nKalman Filter', 'Chang 2014', 'Särkkä 2012', 'Roth 2013'], rotation=-90)

car = visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc="#333333", ec="none")
obserCar = visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc="blue", ec="none", alpha=0.4)
filterCar= visual.animations.Car(state=away_pos, ax=ax, model=model, length=carlength, past=0, predict=0, fc="#9a9a9a", ec="none")#, alpha=0.4)
obserArrow = ax.arrow(*away_pos[:4], width=car.length/4, head_length=car.length/2, head_width=car.length/2, fc="blue", ec="none", alpha=0.4)
ellipse = ax.plot(away_pos[:2], color="red", alpha=0.2)[0]
cancelX = ax.plot([],[], "x", mfc=[0.66,0,0], mec=[0.66,0,0], markersize=carlength*7.5, alpha=0.5)[0]

comment = ax.text(x=midx, y=midy, ha="center", va="center", fontsize=22, s="")
authors = ax.text(x=midx, y=(midy+miny)/2, ha="center", va="center", fontsize=14, s="Jonas Neuhöfer \\hspace{10pt} Jörg Reichardt")
authors.set_color([0,0,0,0])
note = ax.text(x=maxx, y=maxy, ha="right", va="top", fontsize=14, s=f"")


ani = animation.FuncAnimation(fig, func=animate_update_concatenate, frames=range(tot_len), repeat=True)
Writer = animation.writers['ffmpeg']
Writer = Writer(fps=30*upsampling_factor)
ani.save(f"../data/figures/AnimationExplainatory.mp4", writer=Writer, dpi=200)
plt.close(fig)



Animating Multiple robust filters: 100%|██████████| 201/201 [00:17<00:00, 11.69it/s]
Show qualitative results                      : : 5111it [05:02,  7.14it/s]                        